# TMVA Classification Example in Python

<img src="tmva_logo.gif" height="20%" width="20%">

Notebook based on ROOT TMVA Tutorial.

Original author: Lorenzo Moneta | Edited by Sitong An for CMSDAS2019@Pisa

The Toolkit for Multivariate Data Analysis with ROOT (TMVA) is a ROOT-integrated project providing a machine learning environment. For traditional Machine Learning methods like Boosted Decision Trees (BDT), TMVA is widely used by Particle Physics community in the analyses.

This exercise shows an example of TMVA BDT classification using BDTs on some toy data in Python, so as to give you a glimpse on how TMVA is used in analyses. For the same example in C++, and more content on TMVA and ROOT-based multivariate analyses, go to [https://swan.web.cern.ch/content/machine-learning].

This exercise  For  TMVA User Guide [https://github.com/root-project/root/blob/master/documentation/tmva/UsersGuide/TMVAUsersGuide.pdf] can also be a helpful reference in explaining the method parameters.

In [ ]:
import ROOT
from ROOT import TMVA

## Declare Factory

Create the Factory class. Later you can choose the MVA methods you'd like to use. 

The factory is the major TMVA object you have to interact with. Here is the list of parameters you need to pass

 - The first argument is the base name of all the output weight files

 - The second argument is the output file for the training results
  
 - The third argument is a string option defining some general configuration for the TMVA session. For example, all TMVA output can be suppressed by removing the "!" (not) in front of the "Silent" argument in the option string

In [ ]:
ROOT.TMVA.Tools.Instance()
## For PYMVA methods
TMVA.PyMethodBase.PyInitialize();


outputFile = ROOT.TFile.Open("ClassificationOutput.root", "RECREATE")

factory = ROOT.TMVA.Factory("TMVA_Classification", outputFile,
                      "!V:ROC:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" )

## Input Data

We define now the input data file and retrieve the ROOT TTree objects with signal and background input events

In [ ]:
inputFileName = "data/tmva_data.root"

inputFile = ROOT.TFile.Open( inputFileName )

# retrieve input trees

signalTree     = inputFile.Get("sig_tree")
backgroundTree = inputFile.Get("bkg_tree")


and print a summary of the signal tree

In [ ]:
signalTree.Print()

## Declare DataLoader(s)

The next step is to declare the DataLoader class that deals with input data and variables 

We add first the signal and background trees to the data loader, then define the input variables that will be used for the MVA training. Note that you may also use expressions.

In [ ]:
loader = ROOT.TMVA.DataLoader("dataset")

### global event weights per tree (see below for setting event-wise weights)
signalWeight     = 1.0
backgroundWeight = 1.0
   
### You can add an arbitrary number of signal or background trees
loader.AddSignalTree    ( signalTree,     signalWeight     )
loader.AddBackgroundTree( backgroundTree, backgroundWeight )

## Define input variables 

loader.AddVariable("m_jj")
loader.AddVariable("m_jjj")
loader.AddVariable("m_lv")
loader.AddVariable("m_jlv")
loader.AddVariable("m_bb")
loader.AddVariable("m_wbb")
loader.AddVariable("m_wwbb")

## Setup Dataset(s)

Setup the DataLoader by splitting events in training and test samples. 
Here we use a random split and a fixed number of training and test events.


In [ ]:

## Apply additional cuts on the signal and background samples (can be different)
mycuts = ROOT.TCut("")   ## for example:  mycuts = ROOT.TCut("abs(var1)<0.5 && abs(var2-0.5)<1")
mycutb = ROOT.TCut("")   ## for example:  mycutb = ROOT.TCut("abs(var1)<0.5")


loader.PrepareTrainingAndTestTree( mycuts, mycutb,
                                  "nTrain_Signal=7000:nTrain_Background=7000:SplitMode=Random:"
                                   "NormMode=NumEvents:!V" )

# Booking Methods

Here we book the TMVA methods. We book a Likelihood based a BDT and a standard MLP (shallow NN)

In [ ]:
## Boosted Decision Trees
factory.BookMethod(loader,ROOT.TMVA.Types.kBDT, "BDT",
                   "!V:NTrees=200:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:"
                   "BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" )

## Multi-Layer Perceptron (Neural Network)
factory.BookMethod(loader, ROOT.TMVA.Types.kMLP, "MLP",
                   "!H:!V:NeuronType=tanh:VarTransform=N:NCycles=100:HiddenLayers=N+5:TestRate=5:!UseRegulator" );

## Using scikit-learn (optional)

 We can also book some scikit-learn packages into TMVA factory. 
Click on the white space in front of the cell and press Y to activate this. To deactivate again, press R.

## Train Methods

In [ ]:
factory.TrainAllMethods();

## Test  all methods

Here we test all methods using the test data set

In [ ]:
factory.TestAllMethods();   

## Evaluate all methods

Here we evaluate all methods and compare their performances, computing efficiencies, ROC curves etc.. using both training and tetsing data sets. Several histograms are produced which can be examined with the TMVAGui or directly using the output file

In [ ]:
factory.EvaluateAllMethods();

## Plot ROC Curve
We enable JavaScript visualisation for the plots

In [ ]:
%jsroot on

In [ ]:
c1 = factory.GetROCCurve(loader);
c1.Draw();


####  Close outputfile to save all output information (evaluation result of methods)

In [ ]:
outputFile.Close();